## Juliaでpythonのパッケージ(sklearn)を動かしてみる

参考: https://qiita.com/Ag_smith/items/0352890b8cf3420765d0  
https://qiita.com/Miyazaki-Yu/items/51d1e29b1df5ce29bba4  
https://gist.github.com/terasakisatoshi/23b7b69af2893691a992fb7263e185c9  
https://twitter.com/genkuroki/status/1307386185942069248?s=20&t=IQHHqCVKwj0EseVkjN_MdA  
https://qiita.com/Ken-Kuroki/items/115d0679fdce1d15b9d1  

In [1]:
# 初回のインストール
using Pkg
Pkg.add("PyCall")
Pkg.add("Conda")
Pkg.add("Pandas")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("Shuffle")
Pkg.add("Metrics")

using PyCall
using Conda
Conda.update()
Conda.add("numpy")
Conda.add("scikit-learn")
Conda.add("pandas")

ENV["PYTHON"] = ENV["HOME"] * "/.julia/conda/3/bin/python3.10"
Pkg.build("PyCall")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Ch

Solving environment: ...working... done

## Package Plan ##

  environment location: /Users/smamitsu/.julia/conda/3

  added / updated specs:
    - conda


The following packages will be REMOVED:

  ninja-1.11.0-hf86a087_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


[ Info: Running `conda install -y numpy` in root environment


Solving environment: ...working... done

# All requested packages already installed.



[ Info: Running `conda install -y scikit-learn` in root environment


Solving environment: ...working... done

# All requested packages already installed.



[ Info: Running `conda install -y pandas` in root environment


Solving environment: ...working... done

# All requested packages already installed.



    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/6e47d11ea2776bc5627421d59cdcc1296c058071/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/62f417f6ad727987c755549e9cd88c46578da562/build.log`


In [2]:
using PyCall
using CSV
using DataFrames
using Pandas: Pandas, iloc
using Shuffle
using Metrics

### Juliaで読み込んで最小限のところだけpythonにやらせるやり方

In [3]:
data_path = "adult.csv"
jldf = CSV.read(data_path, DataFrame);

In [4]:
# train_test_splitの関数
function partitionTrainTest(data, at = 0.7)
    n = nrow(data)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    data[train_idx,:], data[test_idx,:]
end
train,test = partitionTrainTest(jldf)


(34189×15 DataFrame
   Row │ age      workclass  fnlwgt    education  education-num  marital-statu ⋯
       │ Float64  Int64      Float64   Int64      Float64        Int64         ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │    31.0          4  161153.0         11            9.0                ⋯
     2 │    90.0          4   51744.0         12           14.0
     3 │    21.0          4  155483.0         15           10.0
     4 │    46.0          4  129232.0          9           13.0
     5 │    49.0          1  216453.0          8           11.0                ⋯
     6 │    26.0          4  210521.0         11            9.0
     7 │    29.0          4  150861.0         15           10.0
     8 │    36.0          4  359397.0          1            7.0
     9 │    30.0          4  175931.0         15           10.0                ⋯
    10 │    47.0          5  215620.0         12           14.0
    11 │    42.0          4  228320.0         

In [10]:
# Julia側で分割して、pandasに変換する
X_train = train[:,1:end-1]; y_train = train[:,end]
X_test = test[:,1:end-1]; y_test = test[:,end]

# Pandas.jlの型はそのままPyObject(pandas.DataFrame)に渡すことができる。
X_train_py = Pandas.DataFrame(X_train); y_train_py = Pandas.DataFrame(y_train)
X_test_py = Pandas.DataFrame(X_test); y_test_py = Pandas.DataFrame(y_test)
typeof(X_train_py)  # JuliaのPandas.DataFrameのオブジェクトになるが、これはpythonのpandasに渡すことができる。

Pandas.DataFrame

In [11]:
# pythonに渡してfitさせる
py"""
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

def fit(X,y):
    gbc = GradientBoostingClassifier()
    gbc.fit(X,y)
    return gbc
"""

gbc = py"fit"(X_train_py, y_train_py)

/Users/smamitsu/.julia/conda/3/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


PyObject GradientBoostingClassifier()

In [12]:
# 予測
py"""
def predict(gbc,X):
    y = gbc.predict(X)
    return y
"""

y_pred = py"predict"(gbc, X_test_py)
typeof(y_pred)   # 受け取りはJuliaのオブジェクトになる。

Vector{Int64} (alias for Array{Int64, 1})

In [13]:
# スコアの算出
Metrics.binary_accuracy(y_test,y_pred)

0.8654200505016038

### pythonで読み込んで、処理をする

In [13]:
# pyimportしたpandasでPyObjectとして読み込む
pandas = pyimport("pandas")
pydf = pandas.read_csv(data_path)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39.0,7,77516.0,9,13.0,4,1,1,4,1,2174.0,0.0,40.0,39,1
1,50.0,6,83311.0,9,13.0,2,4,0,4,1,0.0,0.0,13.0,39,1
2,38.0,4,215646.0,11,9.0,0,6,1,4,1,0.0,0.0,40.0,39,1
3,53.0,4,234721.0,1,7.0,2,6,0,2,1,0.0,0.0,40.0,39,1
4,28.0,4,338409.0,9,13.0,2,10,5,2,0,0.0,0.0,40.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39.0,4,215419.0,9,13.0,0,10,1,4,0,0.0,0.0,36.0,39,1
48838,64.0,0,321403.0,11,9.0,6,0,2,2,1,0.0,0.0,40.0,39,1
48839,38.0,4,374983.0,9,13.0,2,10,0,4,1,0.0,0.0,50.0,39,1
48840,44.0,4,83891.0,9,13.0,0,1,3,1,1,5455.0,0.0,40.0,39,1


In [14]:
typeof(pydf)

PyObject

In [15]:
py"""
from sklearn.model_selection import train_test_split
"""
train,test = py"train_test_split"(pydf,test_size=0.7)

2-element Vector{PyObject}:
 PyObject         age  workclass    fnlwgt  ...  hours-per-week  native-country  target
25562  45.0          4   72844.0  ...            46.0              39       1
19831  60.0          4  163729.0  ...            40.0              39       1
17239  33.0          4  351810.0  ...            45.0              26       1
31458  24.0          4  314182.0  ...            50.0              39       1
13571  34.0          4  186824.0  ...            70.0              39       1
...     ...        ...       ...  ...             ...             ...     ...
3816   29.0          2  175262.0  ...            35.0              39       1
39992  39.0          4  111000.0  ...            35.0              39       1
9875   31.0          2  331126.0  ...            48.0              39       1
25165  25.0          4  233461.0  ...            30.0              39       1
31916  18.0          4  218889.0  ...            35.0              39       1

[14652 rows x 15 columns]

In [16]:
py"type"(train)

PyObject <class 'pandas.core.frame.DataFrame'>

In [18]:
# trainはPyObjectなので、pythonから基本的には操作すれば良い。
py"""
import pandas as pd
def X_y(data):
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1]
    return X,y
"""
X_train, y_train = py"X_y"(train)
X_test, y_test = py"X_y"(test)

(PyObject         age  workclass    fnlwgt  ...  capital-loss  hours-per-week  native-country
7169   54.0          4   94055.0  ...           0.0            40.0              39
37781  30.0          4  203488.0  ...           0.0            60.0              39
6228   24.0          4  336088.0  ...           0.0            50.0              39
22027  25.0          4  189850.0  ...           0.0            40.0              39
4887   33.0          1  159548.0  ...           0.0            40.0              39
...     ...        ...       ...  ...           ...             ...             ...
26530  38.0          6   65716.0  ...           0.0            40.0              39
6737   42.0          4   50018.0  ...           0.0            40.0               0
41284  51.0          4  250423.0  ...           0.0            40.0              39
38034  19.0          0  255117.0  ...           0.0            30.0              39
31996  40.0          6  170866.0  ...           0.0            50.

In [50]:
# Pandas.jlでやろうとするとこうなる。
# 末尾を指定できないので、shapeから持ってくる
py"""
def shape(data):
    return data.shape
"""
_, col_num = py"shape"(pydf)

X_train_pd = iloc(Pandas.DataFrame(train))[:,1:col_num-1]
y_train_pd = iloc(Pandas.DataFrame(train))[:,col_num]
X_test_pd = iloc(Pandas.DataFrame(test))[:,1:col_num-1]
y_test_pd = iloc(Pandas.DataFrame(test))[:,col_num]

7169     1
37781    1
6228     1
22027    1
4887     1
        ..
26530    1
6737     1
41284    1
38034    1
31996    0
Name: target, Length: 34190, dtype: int64

In [51]:
py"""
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

def fit_predict_score(X_train,y_train,X_test,y_test):
    gbc = GradientBoostingClassifier()
    gbc.fit(X_train,y_train)
    y_pred = gbc.predict(X_test)
    score = accuracy_score(y_pred, y_test)
    return score
"""
score = py"fit_predict_score"(X_train,y_train,X_test,y_test)

0.8642585551330798

In [52]:
score_pd = py"fit_predict_score"(X_train_pd, y_train_pd, X_test_pd, y_test_pd)

0.8642000584966364